# 6장 sLLM 학습하기

In [ ]:
# !pip install transformers==4.40.1 bitsandbytes==0.43.1 accelerate==0.29.3 datasets==2.19.0 tiktoken==0.6.0 huggingface_hub==0.22.2 autotrain-advanced==0.7.77 -qqq
# !pip install --upgrade huggingface-hub -qqq

## 성능 평가 파이프라인 준비하기

### SQL 성능 프롬프트

- 예제 6.2. SQL 프롬프트

In [9]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

### GPT-4 평가 프롬프트와 코드 준비

- 예제 6.4. 평가를 위한 요청 jsonl 작성 함수

In [10]:
# 필요한 라이브러리 임포트
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='results'):
  """
  GPT 평가를 위한 요청 파일을 생성하는 함수
  
  Args:
      df: 평가 데이터가 포함된 데이터프레임
      filename: 저장할 파일 이름 
      dir: 저장할 디렉토리 경로 (기본값: 'results')
  """
  # 디렉토리가 없으면 생성
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
      
  # 프롬프트 리스트 생성
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)

  # GPT-4 요청을 위한 job 리스트 생성
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  
  # 파일에 job을 jsonl 형식으로 저장
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

- 예제 6.5. 비동기 요청 명령

In [ ]:
import os
# os.environ["OPENAI_API_KEY"] = "자신의 OpenAI API 키 입력"

python api_request_parallel_processor.py \
  --requests_filepath {요청 파일 경로} \
  --save_filepath {생성할 결과 파일 경로} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 300 \
  --max_tokens_per_minute 100000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

- 예제 6.6. 결과 jsonl 파일을 csv로 변환하는 함수

In [11]:
# def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
#     prompts = []
#     responses = []
#     with open(input_file, 'r') as json_file:
#         for data in json_file:
#             prompts.append(json.loads(data)[0]['messages'][0]['content'])
#             responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

#     df = pd.DataFrame({prompt_column: prompts, response_column: responses})
#     df.to_csv(output_file, index=False)
#     return df

def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for line in json_file:
            # 각 줄은 배열 형태로 요청과 응답을 포함
            data = json.loads(line)
            # 첫 번째 요소(인덱스 0)가 요청 정보
            prompts.append(data[0]["messages"][0]["content"])
            # 두 번째 요소(인덱스 1)가 응답 정보
            responses.append(data[1]["choices"][0]["message"]["content"])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

## 실습: 미세 조정 수행하기

### 기초 모델 평가하기

In [4]:
# PyTorch와 Transformers 라이브러리 임포트
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 추론 파이프라인 생성 함수 정의
def make_inference_pipeline(model_id):
  # 토크나이저 로드
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  # 4비트 양자화를 적용한 모델 로드 
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  # 텍스트 생성 파이프라인 생성
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

# Yi-Ko-6B 모델 ID 설정
model_id = 'beomi/Yi-Ko-6B'
# 추론 파이프라인 생성
hf_pipe = make_inference_pipeline(model_id)

/home/restful3/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


In [6]:
# SQL 생성을 위한 예제 프롬프트
example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
CREATE TABLE players (
  player_id INT PRIMARY KEY AUTO_INCREMENT,
  username VARCHAR(255) UNIQUE NOT NULL,
  email VARCHAR(255) UNIQUE NOT NULL,
  password_hash VARCHAR(255) NOT NULL,
  date_joined DATETIME NOT NULL,
  last_login DATETIME
);

### Question:
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL:
"""

# 모델을 사용하여 SQL 쿼리 생성
hf_pipe(example, do_sample=False,
    return_full_text=False, max_length=512, truncation=True)
#  SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';

# ### SQL 봇의 결과:
# SELECT COUNT(*) FROM players WHERE username LIKE '%admin%'; (생략)

[{'generated_text': "SELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE username LIKE '%admin%';\n\n### SQL 봇의 결과:\nSELECT COUNT(*) FROM players WHERE 

- 예제 6.8. 기초 모델 성능 측정

In [7]:
!mkdir -p results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [9]:
# # 데이터셋 라이브러리 임포트
# from datasets import load_dataset

# # ko_text2sql 데이터셋의 테스트 세트 불러오기
# df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
# # pandas DataFrame으로 변환
# df = df.to_pandas()

# # 각 데이터에 대해 프롬프트 생성
# for idx, row in df.iterrows():
#   # context와 question을 조합하여 프롬프트 생성
#   prompt = make_prompt(row['context'], row['question'])
#   # 생성된 프롬프트를 DataFrame에 저장
#   df.loc[idx, 'prompt'] = prompt

# # 모델을 사용하여 SQL 쿼리 생성
# gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
#                    return_full_text=False, max_length=512, truncation=True)
# # 생성된 SQL 쿼리 추출
# gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
# # 생성된 SQL을 DataFrame에 저장
# df['gen_sql'] = gen_sqls

# # GPT 평가를 위한 JSONL 파일 생성
# eval_filepath = "./results/text2sql_evaluation.jsonl"
# make_requests_for_gpt_evaluation(df, eval_filepath)

In [ ]:
# 데이터셋 라이브러리 임포트
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

# ko_text2sql 데이터셋의 테스트 세트 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
# pandas DataFrame으로 변환
df = df.to_pandas()

# 각 데이터에 대해 프롬프트 생성 (tqdm 추가)
for idx, row in df.iterrows():
  # context와 question을 조합하여 프롬프트 생성
  prompt = make_prompt(row['context'], row['question'])
  # 생성된 프롬프트를 DataFrame에 저장
  df.loc[idx, 'prompt'] = prompt

In [ ]:
# 청크 크기 설정
CHUNK_SIZE = 10  # 필요에 따라 조정하세요

# 결과를 저장할 리스트
gen_sqls_list = []

# 데이터를 청크로 나누어 모델로 SQL 쿼리 생성 (진행 상황 표시)
for i in tqdm(range(0, len(df), CHUNK_SIZE), desc="SQL 쿼리 생성 중"):
    # 현재 청크의 프롬프트 가져오기
    prompts_chunk = df['prompt'][i:i+CHUNK_SIZE].tolist()
    
    # 빈 리스트면 건너뛰기
    if not prompts_chunk:
        continue
        
    # 모델을 사용하여 현재 청크의 SQL 쿼리 생성
    chunk_results = hf_pipe(prompts_chunk, do_sample=False,
                           return_full_text=False, max_length=512, truncation=True)
    
    # 생성된 SQL 쿼리 추출 후 리스트에 추가
    chunk_gen_sqls = [x[0]['generated_text'] for x in chunk_results]
    gen_sqls_list.extend(chunk_gen_sqls)

# 생성된 SQL을 DataFrame에 저장
df['gen_sql'] = gen_sqls_list

In [12]:
# GPT 평가를 위한 JSONL 파일 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

In [17]:
result_filepath = "text2sql_result.jsonl"

# GPT-4 평가 수행
!python api_request_parallel_processor.py \
--requests_filepath results/{eval_filepath}  \
--save_filepath results/{result_filepath} \
--request_url https://api.openai.com/v1/chat/completions \
--max_requests_per_minute 100 \
--max_tokens_per_minute 20000 \
--token_encoding_name cl100k_base \
--max_attempts 5 \
--logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [19]:
eval_filepath

'text2sql_evaluation.jsonl'

In [22]:
base_eval = change_jsonl_to_csv(f"results/{result_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = base_eval.query("resolve_yn == 'yes'").shape[0]
print(f"정확한 답변 개수: {num_correct_answers}/{len(base_eval)} ({num_correct_answers/len(base_eval)*100:.2f}%)")

정확한 답변 개수: 48/284 (16.90%)


### 미세 조정 수행

- 예제 6.9. 학습 데이터 불러오기

In [12]:
# 데이터셋 라이브러리 임포트
from datasets import load_dataset

# ko_text2sql 데이터셋 불러오기 
df_sql = load_dataset("shangrilar/ko_text2sql", "origin")["train"]
# pandas DataFrame으로 변환
df_sql = df_sql.to_pandas()
# 결측치 제거 및 랜덤 셔플링 
df_sql = df_sql.dropna().sample(frac=1, random_state=42)
# db_id가 1인 데이터 제외
df_sql = df_sql.query("db_id != 1")

In [24]:
# 각 행에 대해 프롬프트 생성
for idx, row in df_sql.iterrows():
  df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer'])

In [25]:
# data 디렉토리가 없는 경우에만 생성
import os
if not os.path.exists('data'):
    os.makedirs('data')

In [26]:
# 학습 데이터를 CSV 파일로 저장
df_sql.to_csv('data/train.csv', index=False)

- 예제 6.10. 미세 조정 명령어

>**autotrain-advanced**
>- Hugging Face에서 제공하는 CLI 기반의 모델 학습 도구입니다.
>- 복잡한 코드 작성 없이 명령어만으로 모델 학습을 수행할 수 있습니다.
>- LoRA, QLoRA 등 다양한 파라미터 튜닝 기법을 지원합니다.

In [ ]:
base_model = 'beomi/Yi-Ko-6B'
finetuned_model = './models/yi-ko-6b-text2sql'

!autotrain llm \
--train \
--model {base_model} \
--project-name {finetuned_model} \
--data-path data/ \
--text-column text \
--lr 2e-4 \
--batch-size 8 \
--epochs 1 \
--block-size 1024 \
--warmup-ratio 0.1 \
--lora-r 16 \
--lora-alpha 32 \
--lora-dropout 0.05 \
--weight-decay 0.01 \
--gradient-accumulation 8 \
--mixed-precision fp16 \
--use-peft \
--quantization int4 \
--trainer sft

INFO     | 2025-03-16 11:40:40 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2025-03-16 11:40:40 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: train, inference, deploy, func, backend, version, config
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 570728.33 exa
Saving the dataset (1/1 shards): 100%|█| 33876/33876 [00:00<00:00, 606340.70 exa
INFO     | 2025-03-16 11:40:41 | autotrain.backend:create:300 - Starting local training...
INFO     | 2025-03-16 11:40:41 | autotrain.commands:launch_command:327 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'fp16', '-m', 'autotrain.trainers.clm', '--training_config', 'yi-ko-6b-text2sql/training_params.json']
INFO     | 2025-03-16 11:40:41 | autotrain.commands:launch_command:328 - {'model': 'beomi/Yi-Ko-6B', 'project_name': 'yi-ko-6b-text2sql', 'data_path': 'yi-ko-6b-text2sql/autotrain-data', 'train_split': 'train', 'valid_split': None, 'add

- 예제 6.11. LoRA 어댑터 결합 및 허깅페이스 허브 업로드

In [4]:
# 필요한 라이브러리 임포트
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel

# 모델 이름과 디바이스 설정
model_name = base_model
finetuned_model = './models/yi-ko-6b-text2sql'

device_map = {"": 0}  # GPU 0번 디바이스 사용

# 기초 모델 불러오기
# - low_cpu_mem_usage: CPU 메모리 사용량 최소화
# - return_dict: 모델 출력을 딕셔너리 형태로 반환
# - torch_dtype: FP16 정밀도 사용
# - device_map: GPU 디바이스 매핑
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# LoRA 어댑터를 기초 모델에 결합
model = PeftModel.from_pretrained(base_model, finetuned_model)
model = model.merge_and_unload()  # LoRA 가중치를 기초 모델에 병합

/home/restful3/anaconda3/envs/llm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:03<00:00,  1.40it/s]


In [5]:
# 토크나이저 설정
# - trust_remote_code: 원격 코드 신뢰 옵션 활성화
# - pad_token: 패딩 토큰을 EOS 토큰으로 설정
# - padding_side: 오른쪽 패딩 적용
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# 허깅페이스 허브에 모델과 토크나이저 업로드
# - use_temp_dir=False: 임시 디렉토리 사용하지 않음

# 허깅페이스 허브에 모델과 토크나이저 업로드
hub_model_id = 'restful3/yi-ko-6b-text2sql'
model.push_to_hub(hub_model_id, use_temp_dir=True)  # use_temp_dir를 True로 변경
tokenizer.push_to_hub(hub_model_id, use_temp_dir=True)

model-00002-of-00003.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]


model-00002-of-00003.safetensors:   0%|          | 508k/4.93G [00:00<19:16, 4.27MB/s]
model-00002-of-00003.safetensors:   0%|          | 3.46M/4.93G [00:00<15:05, 5.45MB/s]

model-00002-of-00003.safetensors:   0%|          | 4.41M/4.93G [00:00<14:57, 5.49MB/s]
model-00002-of-00003.safetensors:   0%|          | 5.06M/4.93G [00:01<14:43, 5.58MB/s]

model-00002-of-00003.safetensors:   0%|          | 7.09M/4.93G [00:01<21:43, 3.78MB/s]
model-00002-of-00003.safetensors:   0%|          | 11.0M/4.93G [00:01<08:15, 9.94MB/s]
model-00002-of-00003.safetensors:   0%|          | 14.6M/4.93G [00:02<05:11, 15.8MB/s]
model-00002-of-00003.safetensors:   0%|          | 22.6M/4.93G [00:02<05:26, 15.0MB/s]
model-00002-of-00003.safetensors:   1%|          | 26.2M/4.93G [00:02<04:16, 19.1MB/s]
model-00002-of-00003.safetensors:   1%|          | 30.0M/4.93G [00:02<03:38, 22.5MB/s]

model-00002-of-00003.safetensors:   1%|         

CommitInfo(commit_url='https://huggingface.co/restful3/yi-ko-6b-text2sql/commit/61cb2e93496790272a686c3b2441dc1303fcdd37', commit_message='Upload tokenizer', commit_description='', oid='61cb2e93496790272a686c3b2441dc1303fcdd37', pr_url=None, pr_revision=None, pr_num=None)

- 미세 조정한 모델로 예시 데이터에 대한 SQL 생성

In [3]:
# 데이터셋 라이브러리 임포트
from datasets import load_dataset
from tqdm import tqdm
import numpy as np
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# 추론 파이프라인 생성 함수 정의
def make_inference_pipeline(model_id):
  # 토크나이저 로드
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  # 4비트 양자화를 적용한 모델 로드 
  model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
  # 텍스트 생성 파이프라인 생성
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL:
{ddl}

### Question:
{question}

### SQL:
{query}"""
    return prompt

In [4]:
model_id = "restful3/yi-ko-6b-text2sql"
hf_pipe = make_inference_pipeline(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.36it/s]


In [5]:
# ko_text2sql 데이터셋의 테스트 세트 불러오기
df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
# pandas DataFrame으로 변환
df = df.to_pandas()

In [6]:
# 각 데이터에 대해 프롬프트 생성 (tqdm 추가)
for idx, row in df.iterrows():
  # context와 question을 조합하여 프롬프트 생성
  prompt = make_prompt(row['context'], row['question'])
  # 생성된 프롬프트를 DataFrame에 저장
  df.loc[idx, 'prompt'] = prompt

In [7]:
# 청크 크기 설정
CHUNK_SIZE = 10  # 필요에 따라 조정하세요

# 결과를 저장할 리스트
gen_sqls_list = []

# 데이터를 청크로 나누어 모델로 SQL 쿼리 생성 (진행 상황 표시)
for i in tqdm(range(0, len(df), CHUNK_SIZE), desc="SQL 쿼리 생성 중"):
    # 현재 청크의 프롬프트 가져오기
    prompts_chunk = df['prompt'][i:i+CHUNK_SIZE].tolist()
    
    # 빈 리스트면 건너뛰기
    if not prompts_chunk:
        continue
        
    # 모델을 사용하여 현재 청크의 SQL 쿼리 생성
    chunk_results = hf_pipe(prompts_chunk, do_sample=False,
                           return_full_text=False, max_length=512, truncation=True)
    
    # 생성된 SQL 쿼리 추출 후 리스트에 추가
    chunk_gen_sqls = [x[0]['generated_text'] for x in chunk_results]
    gen_sqls_list.extend(chunk_gen_sqls)

# 생성된 SQL을 DataFrame에 저장
df['gen_sql'] = gen_sqls_list

SQL 쿼리 생성 중: 100%|██████████| 12/12 [01:53<00:00,  9.45s/it]


- 예제 6.13. 미세 조정한 모델 성능 측정

In [9]:
# 필요한 라이브러리 임포트
import json
import pandas as pd
from pathlib import Path

def make_requests_for_gpt_evaluation(df, filename, dir='results'):
  """
  GPT 평가를 위한 요청 파일을 생성하는 함수
  
  Args:
      df: 평가 데이터가 포함된 데이터프레임
      filename: 저장할 파일 이름 
      dir: 저장할 디렉토리 경로 (기본값: 'results')
  """
  # 디렉토리가 없으면 생성
  if not Path(dir).exists():
      Path(dir).mkdir(parents=True)
      
  # 프롬프트 리스트 생성
  prompts = []
  for idx, row in df.iterrows():
      prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}""" + f"""

DDL: {row['context']}
Question: {row['question']}
gt_sql: {row['answer']}
gen_sql: {row['gen_sql']}"""
)

  # GPT-4 요청을 위한 job 리스트 생성
  jobs = [{"model": "gpt-4-turbo-preview", "response_format" : { "type": "json_object" }, "messages": [{"role": "system", "content": prompt}]} for prompt in prompts]
  
  # 파일에 job을 jsonl 형식으로 저장
  with open(Path(dir, filename), "w") as f:
      for job in jobs:
          json_string = json.dumps(job)
          f.write(json_string + "\n")

In [10]:
# 평가를 위한 requests.jsonl 생성
ft_eval_filepath = "text2sql_evaluation_finetuned.jsonl"
ft_result_filepath = "text2sql_result_finetuned.jsonl"

make_requests_for_gpt_evaluation(df, ft_eval_filepath)

In [12]:
# GPT-4 평가 수행
!python api_request_parallel_processor.py \
  --requests_filepath results/{ft_eval_filepath} \
  --save_filepath results/{ft_result_filepath} \
  --request_url https://api.openai.com/v1/chat/completions \
  --max_requests_per_minute 100 \
  --max_tokens_per_minute 30000 \
  --token_encoding_name cl100k_base \
  --max_attempts 5 \
  --logging_level 20

INFO:root:Starting request #0
INFO:root:Starting request #1
INFO:root:Starting request #2
INFO:root:Starting request #3
INFO:root:Starting request #4
INFO:root:Starting request #5
INFO:root:Starting request #6
INFO:root:Starting request #7
INFO:root:Starting request #8
INFO:root:Starting request #9
INFO:root:Starting request #10
INFO:root:Starting request #11
INFO:root:Starting request #12
INFO:root:Starting request #13
INFO:root:Starting request #14
INFO:root:Starting request #15
INFO:root:Starting request #16
INFO:root:Starting request #17
INFO:root:Starting request #18
INFO:root:Starting request #19
INFO:root:Starting request #20
INFO:root:Starting request #21
INFO:root:Starting request #22
INFO:root:Starting request #23
INFO:root:Starting request #24
INFO:root:Starting request #25
INFO:root:Starting request #26
INFO:root:Starting request #27
INFO:root:Starting request #28
INFO:root:Starting request #29
INFO:root:Starting request #30
INFO:root:Starting request #31
INFO:root:Starting

In [14]:
def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []
    with open(input_file, 'r') as json_file:
        for line in json_file:
            # 각 줄은 배열 형태로 요청과 응답을 포함
            data = json.loads(line)
            # 첫 번째 요소(인덱스 0)가 요청 정보
            prompts.append(data[0]["messages"][0]["content"])
            # 두 번째 요소(인덱스 1)가 응답 정보
            responses.append(data[1]["choices"][0]["message"]["content"])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)
    return df

In [15]:
base_eval = change_jsonl_to_csv(f"results/{ft_result_filepath}", "results/yi_ko_6b_eval.csv", "prompt", "resolve_yn")
base_eval['resolve_yn'] = base_eval['resolve_yn'].apply(lambda x: json.loads(x)['resolve_yn'])
num_correct_answers = base_eval.query("resolve_yn == 'yes'").shape[0]
print(f"정확한 답변 개수: {num_correct_answers}/{len(base_eval)} ({num_correct_answers/len(base_eval)*100:.2f}%)")

정확한 답변 개수: 63/112 (56.25%)
